In [5]:
from ultralytics import YOLO
import cv2
import os
from functions import *

In [14]:
weight_vehicle_detection_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/weights/vehicle_detection.pt"
tracker_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/weights/tracker/botsort.yaml"
weight_license_plate_detection_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/LicensePlate/src/weights/best.pt"

video0_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/data/IMG_4521.MOV"
video1_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/data/IMG_4526.MOV"
device = "cpu"
class_names = ["car", "motor", "bus", "truck"]
colors = [
    (0, 0, 255),
    (0, 255, 0),
    (255, 0, 0),
    (0, 255, 255) 
]

model_vehicle = YOLO(weight_vehicle_detection_path)
model_vehicle.eval()
model_license_plate = YOLO(weight_license_plate_detection_path)
model_license_plate.eval()
cap = cv2.VideoCapture(video0_path)
fps = cap.get(cv2.CAP_PROP_FPS)
skip = 0.2
skip_frame = max(1, int(fps*skip))
frame_count = 0
scale = 0.4

In [ ]:
# cap = cv2.VideoCapture(video1_path)
# data_path = video1_path.split("/")[-1].split(".")[0]
# os.makedirs(data_path, exist_ok=True)
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     cv2.imwrite(f"{data_path}/{data_path}.png", frame)
#     break
# cap.release()

In [15]:
create_data = SelectAreaDetect(video1_path, scale)
create_data.create_txt()

Point coordinates: (18, 858)
Point coordinates: (358, 101)
Point coordinates: (1075, 75)
Point coordinates: (1533, 446)
Point coordinates: (1532, 861)


In [16]:
mask = gen_mask("IMG_4526/IMG_4526.txt", "IMG_4526/IMG_4526.png", scale)
cap = cv2.VideoCapture(video1_path)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame, frame_pr = process_image(frame, mask, scale)
    vehicle_result = model_vehicle.track(
        frame_pr, 
        conf=0.75, 
        iou=0.7,
        tracker=tracker_path,
        device=device,
        persist=True,
        verbose=False
    )[0]
    for bbox in vehicle_result.boxes:
        if bbox.id is not None:
            x1, y1, x2, y2 = bbox.xyxy[0].cpu().numpy().astype(int)
            id_ = bbox.id[0].cpu().numpy().astype(int)
            cls = bbox.cls[0].cpu().numpy().astype(int)
            color = colors[cls]
            obj = frame[y1:y2, x1:x2]
            try:
                result = model_license_plate.predict(obj, conf=0.8, verbose=False, iou=0.8, device=device)[0].boxes[0]
                lx1, ly1, lx2, ly2 = result.xyxy[0].cpu().numpy().astype(int)
                license = obj[ly1:ly2, lx1:lx2]
                hl, wl = license.shape[:2]
                h, w = obj.shape[:2]
                w_new = int(w/3)
                h_new = int((w_new*hl)/wl)
                license = cv2.resize(license, (w_new, h_new), interpolation=cv2.INTER_AREA)
                frame[y2-h_new:y2, x2-w_new:x2] = license
            except:
                pass
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID:{id_}", (x1, y1+3), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    cv2.imshow("Image", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()